In [1]:
# Import libraries
import pickle
from utils import *
import numpy as np
import matplotlib.pyplot as plt



In [2]:
#open pickle file
path = "data/data_path_dictionary.pkl"
file_names_dict = open_pickle(path)

print("Dictionary contents: ")
file_names_dict



Dictionary contents: 


defaultdict(list,
            {'subject_data': ['svm_data/10004_08693/svm_subj_vecs.mat',
              'svm_data/10008_09924/svm_subj_vecs.mat',
              'svm_data/10009_08848/svm_subj_vecs.mat',
              'svm_data/10016_09694/svm_subj_vecs.mat',
              'svm_data/10017_08894/svm_subj_vecs.mat',
              'svm_data/10018_08907/svm_subj_vecs.mat',
              'svm_data/10021_08839/svm_subj_vecs.mat',
              'svm_data/10022_08854/svm_subj_vecs.mat',
              'svm_data/10023_09126/svm_subj_vecs.mat',
              'svm_data/10027_09455/svm_subj_vecs.mat',
              'svm_data/10033_08871/svm_subj_vecs.mat',
              'svm_data/10034_08879/svm_subj_vecs.mat',
              'svm_data/10035_08847/svm_subj_vecs.mat',
              'svm_data/10036_09800/svm_subj_vecs.mat',
              'svm_data/10037_09903/svm_subj_vecs.mat',
              'svm_data/10038_09063/svm_subj_vecs.mat',
              'svm_data/10039_08941/svm_subj_vecs.mat',
              

In [3]:
masks_path = file_names_dict['mask_data'][0]
subject_path = file_names_dict['subject_data'][0]
file_name = f"data/subject_{file_names_dict['subject_ID'][0]}_data.mat"
file_name_m = f"data/masks.mat"
lbl = file_names_dict['labels'][0]
file_name_lbl = 'labels.mat'



In [7]:
mask_data = access_load_data(masks_path, file_name_lbl)
mask_data.keys()


dict_keys(['__header__', '__version__', '__globals__', 'mask', 'masksubACC', 'masksubAI', 'masksubNAcc', 'masksubmPFC'])

In [8]:
%%time
# Get mat data from aws, download to local then load and save as mat_data
mask_data = access_load_data(masks_path, file_name_m)
mask_data

CPU times: total: 109 ms
Wall time: 895 ms


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Wed Feb 23 12:37:28 2022',
 '__version__': '1.0',
 '__globals__': [],
 'mask': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0

In [14]:
# Create numpy mask
whole_brn_msk = np.ma.make_mask(mask_data['mask'])

# Pull subject data and mask it
subject_data = access_load_data(subject_path, file_name)
subject_data = subject_data['run_01_vec']
subject_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [15]:
subject_data.shape


(144, 592895)

In [13]:
np.array([1,2,3])[np.ma.make_mask(np.array([1,0,1]))]

array([1, 3])

In [ ]:
sub_data = subject_data['run_01_vec']



In [ ]:
whole_brn_msk.shape
wbm = whole_brn_msk.reshape(79*95*79, 1)
wbm.shape



In [ ]:
sub_data.shape

